In [1]:
import PIL
import pandas as pd
import numpy as np
import os 
from tqdm import tqdm
import logging

import matplotlib.image as mpimg
import json

from skimage.color import rgb2gray
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors

import torch
import time
import copy
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as func
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import ToTensor,Resize

from torchvision import models
import torch.nn.functional as F
from torch.optim import lr_scheduler

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import f1_score,accuracy_score,recall_score,classification_report,confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
def load_data(foldername = 'dataset/', train_size = 310, test_size = 110):

    fileList = []
    for dirname, _, filenames in os.walk(foldername):
#         print(dirname)
#         print(filenames)
        for filename in filenames:
            if filename =='666.jpg' or filename == '11702.jpg':
                continue
            fileList.append(os.path.join(dirname, filename))

    
    classList = []

    for i in range(len(fileList)):
        if 'hunzhuo' in fileList[i]:
            classList.append('hunzhuo')
        else:
            classList.append('tuoli')

    df = pd.DataFrame({'filepath':fileList,'label':classList})  

    trainDf,testDf, trainy,testy = train_test_split(df, df.label, train_size = train_size, test_size =test_size)

    return trainDf, testDf

In [3]:
trainDf, testDf = load_data(train_size = 310, test_size = 110)

In [4]:
print(trainDf)

                           filepath    label
70    dataset/hunzhuo/hunzhuo70.jpg  hunzhuo
128  dataset/hunzhuo/hunzhuo128.jpg  hunzhuo
150  dataset/hunzhuo/hunzhuo150.jpg  hunzhuo
122  dataset/hunzhuo/hunzhuo122.jpg  hunzhuo
24    dataset/hunzhuo/hunzhuo24.jpg  hunzhuo
426      dataset/tuoli/tuoli205.jpg    tuoli
4      dataset/hunzhuo/hunzhuo4.jpg  hunzhuo
55    dataset/hunzhuo/hunzhuo55.jpg  hunzhuo
257       dataset/tuoli/tuoli36.jpg    tuoli
157  dataset/hunzhuo/hunzhuo157.jpg  hunzhuo
151  dataset/hunzhuo/hunzhuo151.jpg  hunzhuo
334      dataset/tuoli/tuoli113.jpg    tuoli
206  dataset/hunzhuo/hunzhuo206.jpg  hunzhuo
406      dataset/tuoli/tuoli185.jpg    tuoli
134  dataset/hunzhuo/hunzhuo134.jpg  hunzhuo
102  dataset/hunzhuo/hunzhuo102.jpg  hunzhuo
208  dataset/hunzhuo/hunzhuo208.jpg  hunzhuo
272       dataset/tuoli/tuoli51.jpg    tuoli
175  dataset/hunzhuo/hunzhuo175.jpg  hunzhuo
58    dataset/hunzhuo/hunzhuo58.jpg  hunzhuo
333      dataset/tuoli/tuoli112.jpg    tuoli
182  datas

In [5]:
def getFeat(net, imgList):

    net.eval()
    featList = []

    with torch.no_grad():
        for i in range(len(imgList)):
            feat = net.getFeatures(imgList[i])
            featList.append(feat.squeeze(0).detach().cpu().numpy())

    return np.array(featList)


In [6]:
def getCDDistribution(net,imgList, labelList ):

    uniqueClasses = [0,1]

    NUM_COLORS = 2

    # featList = []

    # for i in range(len(imgList)):
    #     feat = net.getFeatures(imgList[i])
    #     featList.append(feat.squeeze(0).detach().cpu().numpy())


    get_color = plt.get_cmap('viridis')

    colorMap = {}
    for i in range(len(uniqueClasses)):
        colorMap[uniqueClasses[i]] = get_color(i/NUM_COLORS)
    
    support = getFeat(net, imgList)
    X = support

    print(X.shape)

    # scaler = StandardScaler()
    # X = scaler.fit_transform(support)

    pca = PCA(n_components=2)
    pcaX = pca.fit_transform(X=X)
    varExplained = np.sum(pca.explained_variance_ratio_[:2])
    print(f"Variance Explained : {varExplained}")

    colors = [colorMap[int(x)] for x in labelList]
    for i in range(len(uniqueClasses)):
      index = np.array(labelList) == int(uniqueClasses[i])
      plt.scatter(pcaX[index,0], pcaX[index,1], color=colorMap[int(uniqueClasses[i])], label =int(uniqueClasses[i]))
    plt.legend()

    plt.show()

In [7]:
def getSupport(net, dataloader):

    supports = []
    labels = []

    with torch.no_grad():

        net.eval()

        for data in dataloader:
            x,y = data
            x = x.to('cuda')
            feat = net.getFeatures(x)

            supports.append(feat)
            labels.append(y)

    support = torch.cat(supports,dim=0)
    labels = torch.cat(labels,dim=0)

    return support, labels


def compareSamples(index1,index2, metric = 'cosine'):
    img1 = testX[index1].unsqueeze(0).repeat(3,1,1)
    img2 = testX[index2].unsqueeze(0).repeat(3,1,1)

    label1, label2 = testy[index1], testy[index2]

    if label1 ==label2:
        trainLabel = 1
    else:
        trainLabel = 0

    print(f"Labels are {label1} and {label2}")


    feat1 = net.getFeatures(img1.unsqueeze(0).cuda().float())
    feat2 = net.getFeatures(img2.unsqueeze(0).cuda().float())

    if metric=='cosine':
      dist = F.cosine_similarity(feat1, feat2)
    else:
      dist = F.pairwise_distance(feat1,feat2)

    loss = criterion(feat1, feat2, trainLabel)

    print(f"dist is {float(dist)} and loss is {loss}")



def getDistribution(support, labels ):

    uniqueClasses = sorted(pd.Series(labels).unique())

    NUM_COLORS = len(uniqueClasses)

    
    # get_color = matplotlib.colors.LinearSegmentedColormap.from_list(mp, colors=['r', 'y', 'g', 'b'], N=NUM_COLORS)

    get_color = plt.get_cmap('viridis')

    colorMap = {}
    for i in range(len(uniqueClasses)):
        colorMap[uniqueClasses[i]] = get_color(i/NUM_COLORS)
    
    support = support.cpu().numpy()

    scaler = StandardScaler()
    X = scaler.fit_transform(support)

    pca = PCA(n_components=2)
    pcaX = pca.fit_transform(X=X)
    varExplained = np.sum(pca.explained_variance_ratio_[:2])
    print(f"Variance Explained : {varExplained}")

    colors = [colorMap[int(x)] for x in labels]
    for i in range(len(uniqueClasses)):
      index = np.array(labels) == int(uniqueClasses[i])
      plt.scatter(pcaX[index,0], pcaX[index,1], color=colorMap[int(uniqueClasses[i])], label =int(uniqueClasses[i]))
    plt.legend()

    plt.show()


def getClassMap(uniqueClass):

    classMap = {}
    for i in range(len(uniqueClass)):
        classMap[uniqueClass[i]] = i

    classMap_ = {k:v for v,k in classMap.items()}

    return classMap, classMap_

In [8]:
def getPred(net, train_dataloader_, testDataset_, metric = 'cosine', usePCA = False, PCA_components = 10):

    support, labels = getSupport(net, train_dataloader_)

    X = support.cpu().numpy()

    pca = PCA(n_components=PCA_components)
    if usePCA:
      X = pca.fit_transform(X=X)

    knnModel = KNeighborsClassifier(n_neighbors=8,metric=metric)
    knnModel.fit(X, labels)

    labelList = []
    predList = []
    for i in tqdm(range(len(testDataset_))):
        testX = net.getFeatures(testDataset_[i][0].unsqueeze(0).cuda()).detach().cpu().numpy()
        label = int(testDataset_[i][1])
        if usePCA:
          testX = pca.transform(testX)

        pred = knnModel.predict(testX)
        predList.append(pred[0])
        labelList.append(label)

    
    return predList, labelList

In [9]:
def compareDist(net, criterion, img1, img2, label,  metric = 'cosine',printOutput=True):

    net.eval()

    output1 = net.getFeatures(img1)
    output2 = net.getFeatures(img2)

    if metric == 'cosine':
        dist = 1 - torch.nn.functional.cosine_similarity(output1, output2)
    else:
        dist = torch.nn.functional.pairwise_distance(output1, output2)
        
    loss =  criterion(output1, output2, label) 

    dist = dist.detach().cpu().numpy()

    dist = np.round(dist[0],3)

    if printOutput:
        print(f"Dist : {dist} Loss : {loss} Target Label : {int(label)}")

    return dist, loss

In [10]:
def testModel(net, imgList, labelList, testDataset_):

    correct = 0
    net.eval()
    predLabel_list = []
    pred_list = []
    trainFeat = getFeat(net,imgList)
#1
#     from sklearn.svm import SVC
#     knnModel = SVC(gamma='auto', kernel='rbf')

#2     from sklearn.ensemble import RandomForestClassifier
#     knnModel = RandomForestClassifier(n_estimators=10)

#     from sklearn.ensemble import GradientBoostingClassifier
#     knnModel = GradientBoostingClassifier()
    
#4
#     from sklearn.naive_bayes import GaussianNB
 
#     knnModel = GaussianNB()
#5
    knnModel = KNeighborsClassifier(n_neighbors=10, metric='cosine')
    knnModel.fit(trainFeat, labelList)

    for i in tqdm(range(len(testDataset_))):
        predImg, predLabel = testDataset_[i]
        predImg = predImg.cuda().unsqueeze(0)
        predLabel_list.append(predLabel)
    
        testFeat = getFeat(net, [predImg])
        pred = knnModel.predict(testFeat)
        pred_list.extend(pred)
#         print(predLabel)
#         print(pred)
        if pred == predLabel:
            correct += 1
#     print(predLabel_list)
#     print(pred_list)
    class_report=classification_report( predLabel_list, pred_list, target_names=['class 0', 'class 1'])
    # print(f"Accuracy is : {correct/len(testDataset_)}")
    print(class_report)
    acc = np.round(correct/len(testDataset_),4)
    return acc

In [11]:
def predictModel(net, test_dataloader_):

    net.eval()

    labelList = []
    predList = []

    with torch.no_grad():

        for (img,labels) in test_dataloader_:
            img,labels = img.cuda(), labels.cuda()
            
            outputs = net(img)
            _, preds = torch.max(outputs, 1)

            labelList = labelList + list(labels.cpu().numpy())
            predList = predList + list(preds.cpu().numpy())

    return labelList, predList

In [12]:
class SiameseDataset(torch.utils.data.Dataset):
  def __init__(self, df, prob = 0.5, transform = None, classMap= None, classMap_ = None):
#     """ 
#     df : Dataframe with filename and label columns.


#     """
    
    self.df = df
    self.transform = transform
    self.numClasses = df.label.nunique()
    self.uniqueLabels = sorted(df.label.unique())
    self.prob = prob

    if classMap is None or classMap_ is None:
        self.classMap, self.classMap_ = getClassMap(self.df.label.unique())
    else:
        self.classMap, self.classMap_ = classMap, classMap_
        
  def __len__(self):
    
    return len(self.df)
    
  def __getitem__(self, index):

    is_positive = np.random.random() < self.prob

    if is_positive:
        posLabel = np.random.choice(self.uniqueLabels,1)[0]

        # print(posLabel)
        subDf = self.df[self.df.label == posLabel] 
        subDf = subDf.sample(2, replace = False)
            
        filename1, filename2 = subDf.iloc[0]['filepath'], subDf.iloc[1]['filepath']
        img1, img2 =  Image.open(os.path.join(filename1)).convert('RGB'), Image.open(os.path.join(filename2)).convert('RGB')
        
        target_label = 1
        
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            
        # img1 = img1/255.0
        # img2 = img2/255.0

    else:

        negLabels = np.random.choice(self.uniqueLabels,2, replace = False)


        subDf1 = self.df[self.df.label == negLabels[0]].sample(1)
        subDf2 = self.df[self.df.label == negLabels[1]].sample(1)

        filename1, filename2 = subDf1.iloc[0]['filepath'], subDf2.iloc[0]['filepath']
        img1, img2 =  Image.open(os.path.join(filename1)).convert('RGB'), Image.open(os.path.join(filename2)).convert('RGB')

        target_label = 0

        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            
        # img1 = img1/255.0
        # img2 = img2/255.0


    
    return img1,img2, target_label

In [13]:
class ImageDataset(torch.utils.data.Dataset):
  def __init__(self, df, transform, classMap= None, classMap_ = None):
    
    
    self.df = df
    self.transform = transform

    if classMap is None or classMap_ is None:
        self.classMap, self.classMap_ = getClassMap(self.df.label.unique())
    else:
        self.classMap, self.classMap_ = classMap, classMap_
        
    
  def __len__(self):
    
    return len(self.df)
    
  def __getitem__(self, index):
    
    filename = self.df.iloc[index]['filepath']
    
    img =  Image.open(os.path.join(filename)).convert('RGB')

    
    label = self.classMap[self.df.iloc[index]['label']]
    
    if self.transform is not None:
        img = self.transform(img)
    
    
    # img = img/255.0
    
    return img, label



In [14]:
class ModuleHook():
    def __init__(self, module, backward=True):

        self.gradList = []

        if backward==False:
            self.hook = module.register_forward_hook(self.hook_fn)
        else:
            self.hook = module.register_backward_hook(self.hook_fn)
    
    def __getitem__(self,key):
        return self.gradList[key]

    def hook_fn(self, module, grad_input, grad_output):
        self.gradList.append(grad_output.cpu().numpy())

    def close(self):
        self.hook.remove()


class ParamHook():
    def __init__(self, param):
        self.gradList = []
        self.hook = param.register_hook(self.hook_fn)

    def __getitem__(self, key):
        return self.gradList[key]
      
    def hook_fn(self, grad):
        meanGrad = grad.abs().mean().cpu().numpy()
        maxGrad = grad.abs().max().cpu().numpy()
        self.gradList.append([meanGrad,maxGrad])

    def close(self):
        self.hook.remove()


class gradCapture():

   def __init__(self):
     self.paramGradList = {}
     self.gradList = {}

     self.paramHooks = {}
     self.moduleHooks = {}


   def registerHooks(self, model):

     for name, param in net.named_parameters():
       self.paramHooks[name] = ParamHook(param)

   def plotGrad(self, paramList, plotMean = True, plotRange = None):

     length = len(self.paramHooks[list(self.paramHooks.keys())[0]].gradList)
     
     if plotRange :
         x = list(range(plotRange[0], plotRange[1]))
     else:
         x = list(range(length))

     for i in range(len(paramList)):
       key = paramList[i]
       if plotMean:
         y = [i[0] for i in gradCap.paramHooks[key].gradList]
       else:
         y = [i[1] for i in gradCap.paramHooks[key].gradList]

       if plotRange :
        y = np.array(y)[plotRange[0]:plotRange[1]]
    
        
       plt.plot(x,y,label=key)

     plt.legend()

In [15]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=15.0, metric='cosine'):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
        self.metric = metric

    def forward_cosine(self, output1, output2, label):
        euclidean_distance = 1 - torch.nn.functional.cosine_similarity(output1, output2)
        # print(euclidean_distance)
        loss_contrastive = torch.mean((label) * torch.pow(euclidean_distance, 2) +
                                      (1 - label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        
        return loss_contrastive
      
    def forward_euclidean(self, output1, output2, label):
        euclidean_distance = torch.nn.functional.pairwise_distance(output1, output2)
#         euclidean_distance =euclidean_distance.to(output1.device)
        label = label.float()
        euclidean_distance =euclidean_distance.float()
        a = (label) * torch.pow(euclidean_distance, 2)
        b = (1 - label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2)
        loss_contrastive =torch.mean( a + b)
        return loss_contrastive

    def forward(self, output1, output2, label):

      if self.metric =='cosine':
        loss = self.forward_cosine(output1, output2, label)
      elif self.metric=='euclidean':
        loss = self.forward_euclidean(output1, output2, label)
      
      return loss

In [16]:
train_transform = transforms.Compose([
      transforms.Resize([224,224]),
     transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
#      transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
     transforms.RandomHorizontalFlip(),#随机水平翻转图片
     transforms.RandomVerticalFlip(),#随机垂直翻转图片
     transforms.RandomRotation(degrees=30),#随机旋转30°
#      transforms.RandomCrop(224,224),
#     transforms.RandomErasing(p=0.5, 
#                             scale=(0.02, 0.33), 
#                             ratio=(0.3, 3.3), 
#                             value=0, 
#                             inplace=False),

     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
                                     transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [18]:
NUM_TRAIN_SAMPLES = 310
NUM_TEST_SAMPLES = 110
STEPS_PER_EPOCH = 250
NUM_EPOCHS =100
EMBEDDING_SIZE = 100

In [19]:
class ChannelAttention(nn.Module):
    def __init__(self, in_planes=512, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc1   = nn.Conv2d(in_planes, in_planes // 16, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2   = nn.Conv2d(in_planes // 16, in_planes, 1, bias=False)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

In [20]:
ca =ChannelAttention()

In [21]:
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=3):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

In [22]:
sa = SpatialAttention()

In [23]:
vgg_16 = models.vgg16(pretrained=False)
pthfile = r'vgg16-397923af.pth'
vgg_16.load_state_dict(torch.load(pthfile))


In [24]:
vgg_feature = vgg_16.features

In [25]:
print(vgg_feature)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(

In [26]:
# for parameters in vgg_feature.parameters():
#     print(parameters)

In [27]:
class vgg16_Siamese(torch.nn.Module):
    def __init__(self):

        super(vgg16_Siamese, self).__init__()
        
        self.vgg_features = vgg_feature
        self.activation = nn.ReLU()
        self.ca = ca
        self.sa =sa
        self.fc = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 1000),
            
        )

        
    def getFeatures(self,x):

        x = self.vgg_features(x)
#         x = self.ca(x)*x
        x = self.sa(x)*x
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    def forward(self,x1, x2):

        x1 = self.getFeatures(x1)
        x2 = self.getFeatures(x2)

        return x1, x2

In [28]:
net=vgg16_Siamese()

In [29]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)
        # optimizer = nn.DataParallel(optimizer)

net.to(device) #加入此句，可以解决相应问题


vgg16_Siamese(
  (vgg_features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  

In [30]:

gradCap = gradCapture()
gradCap.registerHooks(net)

criterion = ContrastiveLoss(margin=0.4, metric='euclidean')

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()) ,lr = 0.00001 )

In [31]:
trainDf,testDf = load_data(train_size = NUM_TRAIN_SAMPLES, test_size = NUM_TEST_SAMPLES)

trainDataset = SiameseDataset(trainDf, prob = 0.5, transform=train_transform)

trainDataset_ = ImageDataset(trainDf, transform = train_transform)
testDataset_ = ImageDataset(testDf, transform = test_transform, classMap = trainDataset_.classMap, classMap_= trainDataset_.classMap_)

train_dataloader = DataLoader(trainDataset,
                        shuffle=True,
                         num_workers=0,
                        batch_size=50)

train_dataloader_ = DataLoader(trainDataset_,
                        shuffle=False,
                         num_workers=0,
                        batch_size=50)

test_dataloader_ = DataLoader(testDataset_,
                        shuffle=False,
#                         num_workers=1,
                        batch_size=200)

In [32]:
indexes = list(range(NUM_TRAIN_SAMPLES))

trainImgList = []
trainLabelList = []

for i in range(len(indexes)):
    x, y = trainDataset_[indexes[i]]
    trainImgList.append(x)
    trainLabelList.append(y)
    trainImgList[i] = trainImgList[i].cuda().unsqueeze(0)

In [33]:
net.train()

for epoch in range(NUM_EPOCHS):
    epoch_loss = []
#     support, labels = getSupport(net, train_dataloader_)
#     getDistribution(support, labels)
#     plt.show()

    for i, data in enumerate(train_dataloader,0):
        img0, img1 , label = data
        img0, img1 , label = img0.cuda(), img1.cuda() , label.cuda()
 
        optimizer.zero_grad()
        output1,output2 = net(img0,img1)
        loss_contrastive = criterion(output1,output2,label)
        print(loss_contrastive)
        loss_contrastive.mean().backward()
        optimizer.step()

        epoch_loss.append(loss_contrastive.item())


    print(f"Epoch Loss : {np.mean(epoch_loss)}")


# support, labels = getSupport(net, train_dataloader_)
# getDistribution(support, labels)
# plt.show()

tensor(21.3841, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(14.8970, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(10.8602, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(9.7862, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(6.3571, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(6.0038, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(4.7869, device='cuda:0', grad_fn=<MeanBackward1>)
Epoch Loss : 10.582179818834577
tensor(2.9957, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(2.7091, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(2.8345, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.8297, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.5480, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.2085, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(1.2665, device='cuda:0', grad_fn=<MeanBackward1>)
Epoch Loss : 2.055987630571638
tensor(1.0996, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.9964, device='cuda:0', grad_fn=<MeanBackward1>)
tensor(0.8529, device=

In [34]:
testAcc = testModel(net,trainImgList, trainLabelList, testDataset_)
print(f" Test Acc : {testAcc}")

100%|██████████| 110/110 [00:14<00:00,  7.62it/s]

              precision    recall  f1-score   support

     class 0       0.57      0.88      0.69        57
     class 1       0.70      0.30      0.42        53

   micro avg       0.60      0.60      0.60       110
   macro avg       0.64      0.59      0.56       110
weighted avg       0.63      0.60      0.56       110

 Test Acc : 0.6


第一次

weighted avg       0.86      0.85      0.85       110

 Test Acc : 0.8545
 
第二次

weighted avg       0.86      0.85      0.85       110

 Test Acc : 0.8545
 
第三次

weighted avg       0.94      0.94      0.94       110

 Test Acc : 0.9364

第四次
weighted avg       0.86      0.85      0.85       110

 Test Acc : 0.8545
第五次

weighted avg       0.86      0.85      0.85       110

 Test Acc : 0.8545

accuracy：0.8700

precision：0.876

recall：0.868

f1：0.868
